In [ ]:
!pip install openai-whisper==20231117
!pip install openai==1.31.0
!pip install gradio==3.40.0
!pip install pytubefix==7.1.3

# SRT 자막 파일 만들기 - whisper 라이브러리 활용

STT(Speech-To-Text)

In [ ]:
import whisper

model = whisper.load_model("large")

audio_file_name = "audio_from_youtube.webm"

transcript = model.transcribe(audio_file_name)

transcript

변환된 Text 저장

In [ ]:
srt_writer = get_writer(output_format="srt", output_dir=".")
srt_writer(transcript, audio_file_name)

Gradio 연결

In [ ]:
import os

import gradio as gr
import whisper
from pytubefix import YouTube
from whisper.utils import get_writer

class WhisperInferencer:
    def __init__(self):
        self.model = whisper.load_model("large")
        self.srt_writer = get_writer(output_format="srt", output_dir=".")

    def inference(self, audio_file_path: str) -> str:
        transcript = self.model.transcribe(audio_file_path)
        self.srt_writer(transcript, audio_file_path)

        filename = os.path.basename(audio_file_path)
        filename = filename.split(".")[0]

        return f"{filename}.srt"

whipser_inferencer = WhisperInferencer()

def transcribe(link: str):
    video_file_name = "video_from_youtube.mp4"
    audio_file_name = "audio_from_youtube.webm"
    yt = YouTube(link)

    # Extract video
    streams = yt.streams.filter(progressive=True, file_extension="mp4", type="video").order_by("resolution").desc()
    streams[0].download(filename=video_file_name)

    # Extract audio
    audio_streams = yt.streams.filter(type="audio").order_by("abr").desc()
    audio_streams[0].download(filename=audio_file_name)

    transcript_file = whipser_inferencer.inference(audio_file_name)
    return transcript_file, [video_file_name, transcript_file]

# Set gradio app
with gr.Blocks() as app:
    gr.Markdown("# Youtube 자막 생성기")

    with gr.Row():
        with gr.Column(scale=1):
            link = gr.Textbox(label="Youtube Link")
            subtitle = gr.File(label="Subtitle", file_types=[".srt"])
            transcribe_btn = gr.Button(value="자막 생성!")

        with gr.Column(scale=4):
            output_video = gr.Video(label="Output", height=500)

    transcribe_btn.click(transcribe, [link], [subtitle, output_video])

app.launch(inline=False, share=True)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.40.0, however version 5.0.1 is available, please upgrade.
--------
Running on public URL: https://7cfc032775c5a128f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
app.close()

Closing server running on port: 7860
